## Test for testing baseline methods
### Supervised: SVM, RF, XGBOOST
### Semi-supervised: TSVM

In [7]:
import numpy as np
import visdom
import utils
import seaborn as sns
from datasets import get_patch_data, get_pixel_idx, HyperX_patches
import os
from sklearn import preprocessing
from sklearn.decomposition import PCA
import random
import math
from matplotlib import pyplot as plt
import torch.utils.data as data

In [139]:
vis = visdom.Visdom()

Setting up a new session...


In [2]:
data_path = '/home/oscar/Desktop/Exjobb/Data/ieee_supplement/Hyperspectral_Grids/Salinas/'

label_values = ["Undefined", "Brocoli_green_weeds_1", "Brocoli_green_weeds_2", "Fallow",
                        "Fallow_rough_plow", "Fallow_smooth", "Stubble",
                        "Celery", "Grapes_untrained", "Soil_vinyard_develop",
                        "Corn_senesced_green_weeds", "Lettuce_romaine_4wk", "Lettuce_romaine_5wk",
                        "Lettuce_romaine_6wk", "Lettuce_romaine_7wk", "Vinyard_untrained",
                        "Vinyard_vertical_trellis"]

palette = None
rgb_bands = (43, 21, 11)

if palette is None:
    # Generate color palette
    palette = {0: (0, 0, 0)}
    for k, color in enumerate(sns.color_palette("hls", len(label_values) - 1)):
        palette[k + 1] = tuple(np.asarray(255 * np.array(color), dtype='uint8'))
invert_palette = {v: k for k, v in palette.items()}

def convert_to_color(x):
    return utils.convert_to_color_(x, palette=palette)
def convert_from_color(x):
    return utils.convert_from_color_(x, palette=invert_palette)

In [3]:
fold = 4

train_img, train_gt, test_img, test_gt, label_values, ignored_labels, rgb_bands, palette = get_patch_data('Salinas', 5, target_folder=data_path, fold=fold)

if palette is None:
    # Generate color palette
    palette = {0: (0, 0, 0)}
    for k, color in enumerate(sns.color_palette("hls", len(label_values) - 1)):
        palette[k + 1] = tuple(np.asarray(255 * np.array(color), dtype='uint8'))
invert_palette = {v: k for k, v in palette.items()}

def convert_to_color(x):
    return utils.convert_to_color_(x, palette=palette)
def convert_from_color(x):
    return utils.convert_from_color_(x, palette=invert_palette)

n_bands = train_img.shape[-1]
n_classes = len(label_values) - len(ignored_labels)

idx_sup, idx_val, idx_unsup = get_pixel_idx(train_img, train_gt, ignored_labels, 5)

In [18]:
hyperparams = {'patch_size': 5, 'dataset': 'Salinas', 'ignored_labels': ignored_labels, 
               'flip_augmentation': False, 'radiation_augmentation': False, 'mixture_augmentation': False,
              'center_pixel': True, 'supervision': 'full', 'pca_augmentation': False, 'pca_strength': 1, 'cutout_spatial': False,
              'cutout_spectral': False, 'augmentation_magnitude': 1, 'spatial_combinations': False, 'spectral_mean': False,
              'moving_average': False}

val_dataset = HyperX_patches(train_img, train_gt, idx_val, labeled=True, **hyperparams)
val_loader = data.DataLoader(val_dataset, batch_size=10)

train_labeled_dataset = HyperX_patches(train_img, train_gt, idx_sup, labeled=True, **hyperparams)
train_labeled_loader = data.DataLoader(train_labeled_dataset, batch_size=len(idx_sup),
                                       #pin_memory=True, num_workers=5,
                                       shuffle=True, drop_last=True)

unlabeled_ratio = math.ceil(len(idx_unsup)/len(idx_sup))

train_unlabeled_dataset = HyperX_patches(train_img, train_gt, idx_unsup, labeled=False, **hyperparams)
train_unlabeled_loader = data.DataLoader(train_unlabeled_dataset, batch_size=len(idx_unsup),
                                       #pin_memory=True, num_workers=5,
                                       shuffle=True, drop_last=True)
amount_labeled = idx_sup.shape[0]

Move data to simple list with pixel form

In [91]:
i = 0
X = np.zeros((len(idx_sup), n_bands))
Y = np.zeros(len(idx_sup))
for p, x, y in idx_sup:
    X[i, :] = train_img[p,x,y]
    Y[i] = train_gt[p,x,y] - 1
    i += 1

In [92]:
i = 0
X_un = np.zeros((len(idx_unsup), n_bands))
for p, x, y in idx_unsup:
    X_un[i, :] = train_img[p,x,y]
    i += 1

In [93]:
results = []

### SVM

In [8]:
import sklearn.svm as SVM

In [124]:
SVM_GRID_PARAMS = [{'kernel': ['rbf'], 'gamma': [5e-1, 2e-1], 'C': [1000, 1500]}]

class_weight = None
clf = SVM.SVC(class_weight=class_weight, kernel='rbf', gamma=0.5, C=1000)
#clf = sklearn.model_selection.GridSearchCV(clf, SVM_GRID_PARAMS, verbose=5, n_jobs=4)
clf.fit(X, Y)
#print('SVMs best params: {} '.format(clf.best_params_))
#save_model(clf, 'SVM', 'Salinas')

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.5, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [46]:
prediction = clf.predict(test_img.reshape(-1, n_bands))
prediction = prediction.reshape(test_img.shape[:2])

In [127]:
clf.support_vectors_

array([[0.03432621, 0.05503267, 0.07518547, ..., 0.00188241, 0.00121803,
        0.00166095],
       [0.04229875, 0.06355885, 0.06798804, ..., 0.00221459, 0.00155022,
        0.00132876],
       [0.0428524 , 0.05636142, 0.08271509, ..., 0.00221459, 0.00132876,
        0.00121803],
       ...,
       [0.04240948, 0.05458974, 0.06123353, ..., 0.00321116, 0.00177168,
        0.00221459],
       [0.03554424, 0.0376481 , 0.06101207, ..., 0.00243605, 0.00177168,
        0.00132876],
       [0.03399402, 0.03665153, 0.05979404, ..., 0.00287897, 0.00199313,
        0.00199313]])

Connection is already closed.
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 11

[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 1

[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 1

[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 1

### Random Forest

In [51]:
from sklearn.ensemble import RandomForestClassifier
import sklearn

In [81]:
#RF_GRID_PARAMS = [{'max_depth': [32, 256, 512], 'n_estimators': [100, 200, 400]}]

clf = RandomForestClassifier(n_estimators = 200, min_samples_split=2, \
                    max_features=10, max_depth=10)
#clf = sklearn.model_selection.GridSearchCV(clf, RF_GRID_PARAMS, verbose=5, n_jobs=4)
clf.fit(X, Y)
#print('RFs best params: {} '.format(clf.best_params_))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features=10, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [82]:
prediction = clf.predict(test_img.reshape(-1, n_bands))
prediction = prediction.reshape(test_img.shape[:2])

### XGBoost

In [91]:
from xgboost import XGBClassifier

In [122]:
clf = XGBClassifier(n_estimators=200, max_depth=10, objective='multi::softmax', num_class=n_classes)
clf.fit(X, Y)

prediction = clf.predict(test_img.reshape(-1, n_bands))
prediction = prediction.reshape(test_img.shape[:2])

### TSVM

In [15]:
from scipy.optimize import minimize

Initialize parameters

In [35]:
nb_labeled = len(X)
nb_unlabeled = len(X_un)
nb_samples = nb_labeled + nb_unlabeled

n_bands = X.shape[-1]

X_t = np.concatenate((X, X_un))
Y_t = np.concatenate((Y, -1*np.ones(nb_unlabeled)))

w = np.random.uniform(-0.1, 0.1, size=n_bands)
eta_labeled = np.random.uniform(0.0, 0.1, size=nb_labeled)
eta_unlabeled = np.random.uniform(0.0, 0.1, size=nb_unlabeled)
y_unlabeled = np.random.uniform(-1.0, 1.0, size=nb_unlabeled)
b = np.random.uniform(-0.1, 0.1, size=1)

C_labeled = 1.0
C_unlabeled = 10.0

theta0 = np.hstack((w, eta_labeled, eta_unlabeled, y_unlabeled, b))

Define objective function

In [46]:
def svm_target(theta, Xd, Yd):
    wt = theta[0:n_bands].reshape((n_bands, 1))
    s_eta_labeled = np.sum(theta[n_bands:n_bands+nb_labeled])
    s_eta_unlabeled = np.sum(theta[n_bands+nb_labeled:n_bands+nb_samples])
    return (C_labeled*s_eta_labeled) + (C_unlabeled*s_eta_unlabeled) + (0.5*np.dot(wt.T, wt))

Labeled and unlabeled constraints

In [47]:
def labeled_constraint(theta, Xd, Yd, idx):
    wt = theta[0:n_bands].reshape((n_bands, 1))
    c = Yd[idx] * (np.dot(Xd[idx], wt) + theta[-1]) + \
    theta[n_bands:n_bands+nb_labeled][idx] - 1.0
    return (c>=0)[0]

def unlabeled_constraint(theta, Xd, idx):
    wt = theta[0:n_bands].reshape((n_bands, 1))
    c = theta[n_bands+nb_labeled:n_bands+nb_samples+nb_unlabeled][idx-nb_samples+nb_unlabeled]*\
    (np.dot(Xd[idx],wt) + theta[-1]) + \
    theta[n_bands+nb_labeled:n_bands+nb_samples][idx-nb_samples+nb_unlabeled] - 1.0
    return (c>=0)[0]

Slack constraints

In [48]:
def eta_labeled_constraint(theta, idx):
    return theta[n_bands:n_bands+nb_labeled][idx] >= 0

def eta_unlabeled_constraint(theta, idx):
    return theta[n_bands+nb_labeled:n_bands+nb_samples][idx-nb_samples+nb_unlabeled] >= 0

def y_constraint(theta, idx):
    return np.power(theta[n_bands+nb_samples:n_bands+nb_samples+nb_unlabeled][idx], 2) == 1.0

SVM constraints for scipy

In [49]:
svm_constraints = []

for i in range(nb_labeled):
    svm_constraints.append({
        'type': 'ineq',
        'fun': labeled_constraint,
        'args': (X, Y, i)
    })
    svm_constraints.append({
        'type': 'ineq',
        'fun': eta_labeled_constraint,
        'args': (i, )
    })
    
for i in range(nb_labeled, nb_samples):
    svm_constraints.append({
        'type': 'ineq',
        'fun': unlabeled_constraint,
        'args': (X, i)
    })
    svm_constraints.append({
        'type': 'ineq',
        'fun': eta_unlabeled_constraint,
        'args': (i, )
    })
    
for i in range(nb_unlabeled):
    svm_constraints.append({
        'type': 'eq',
        'fun': y_constraint,
        'args': (i,)
    })

Run scipy to optimze this

In [50]:
result=minimize(fun=svm_target, 
               x0=theta0,
               constraints=svm_constraints,
               args=(X_t,Y_t),
               method='SLSQP',
               tol=0.0001,
               options={'maxiter': 1000}
               )

IndexError: index 4656 is out of bounds for axis 0 with size 4656

To be able to perform multiclass classification we need to create n_classes amount of TSVM's and train each on classifying if a sample belongs to one class or not. These will then need to perform a winner-takes-it-all approach to choose which class a sample belongs to. We also need to transform the linear kernel to a gaussian kernel. It is also benefitial to instead move to Lagrangian optimization.

In [ ]:
def tsvm_target(theta, Xd, Yd):
    wt = theta[0:n_bands].reshape((n_bands, 1))
    s_eta_labeled = np.sum(theta[n_bands:n_bands+nb_labeled])
    s_eta_unlabeled = np.sum(theta[n_bands+nb_labeled:n_bands+nb_samples])
    return (C_labeled*s_eta_labeled) + (C_unlabeled*s_eta_unlabeled) + (0.5*np.dot(wt.T, wt))

TSVM implementation from Bruzzone

In [146]:
import importlib
import qns3vm
import scipy
import sklearn.svm as SVM 

qns3vm = importlib.reload(qns3vm)

from qns3vm import QN_S3VM

import warnings
warnings.filterwarnings("ignore", category=PendingDeprecationWarning) 

In [93]:
G = 10

C_labeled = 1.0
C_unlabeled = np.zeros(G)
C_unlabeled[0] = C_labeled/(10*G)
C_max = 10.0

sigma = 1.0
Np = 0
Nm = 0
A = np.zeros(n_classes)

rand_generator = random.Random()

CLF = []

for i in range(n_classes):
    idx_class = Y == i
    idx_rest = Y != i
    X_train = np.concatenate((X[idx_class], X[idx_rest]))
    Y_train = np.concatenate((np.ones(len(X[idx_class])).astype(int), -np.ones(len(X[idx_rest])).astype(int)))
    # Classifier is a transductive SVM
    #CLF.append(QN_S3VM(X_train.tolist(), Y_train.tolist(), X_un.tolist(), rand_generator, lam=C_labeled, lamU=C_unlabeled[0], kernel_type='RBF'))
    #CLF[i].train()
    
    # Classifier is a standard SVM
    CLF.append(SVM.SVC(kernel='rbf', gamma=0.5, C=C_labeled)) #C=1000 other option
    CLF[i].fit(X_train,Y_train)
    
    support = CLF[i].n_support_
    
    Np = support[1]
    Nm = support[0]
    
    A[i] = np.min((Np,Nm))        

In [122]:
for g in range(G):
    for i in range(n_classes):
        #Find A transductive samples
        values = CLF[i].decision_function(X_un)
        values = np.asarray(values)
        
        if np.max(values)>0 and np.min(values)<0:
            #Extract values of positive unlabeled samples
            values_p = np.abs(1 - values[values>0])
            idx_p = values_p.argsort()[:int(A[i])]
            #Threshold
            D_p = np.sum(values[idx_p])/A[i]
            Th_p = D_p*np.max(np.abs(values[idx_p]))
            #Trim candidate set
            N_p = len(values[idx_p][np.abs(values[idx_p])>=Th_p])
        
        
            values_m = np.abs(-1 - values[values<0])
            idx_m = values_m.argsort()[:int(A[i])]
            #Threshold
            D_m = np.sum(values[idx_m])/A[i]
            Th_m = D_m*np.max(np.abs(values[idx_m]))
            #Trim candidate set
            N_m = len(values[idx_m][np.abs(values[idx_m])>=Th_m])
            
            N = np.min((N_p, N_m))
        
            if N == N_p:
                X_un_p = X_un[idx_p]
                X_un_m = X_un[idx_m[:N]]
                del_idx = np.concatenate((idx_p, idx_m[:N]))
            elif N==N_m:
                X_un_m = X_un[idx_m]
                X_un_p = X_un[idx_p[:N]]
                del_idx = np.concatenate((idx_m, idx_p[:N]))
               
            #Update datasets               
            idx_class = Y == i
            idx_rest = Y != i
            X_train = np.concatenate((X[idx_class], X[idx_rest]))
            Y_train = np.concatenate((np.ones(len(X[idx_class])), -np.ones(len(X[idx_rest]))))
            
            np.append(X_train, (X_un_m, X_un_p))
            np.append(Y_train, (-np.ones(len(X_un_m)), np.ones(len(X_un_p))))
            X_un = np.delete(X_un, del_idx,0)
               
            #Update weight factor        
            #C[i] = (C_max - C[0])/G^2*g^2 + C[0]
        
            #Retrain the TSVM
            CLF[i].fit(X_train, Y_train)
    
    

In [124]:
pred_values = []
for i in range(n_classes):
    pred_values.append(CLF[i].decision_function(test_img.reshape(-1, n_bands)))

In [140]:
predicted_values = np.asarray(pred_values)
prediction = np.argmax(predicted_values, axis=0)
prediction = prediction.reshape(test_img.shape[:2])

In [136]:
np.argmax(predicted_values, axis=0)

array([3, 3, 3, ..., 3, 3, 3])

## Run results

In [141]:
run_results = utils.metrics(prediction, test_gt, ignored_labels=ignored_labels, n_classes=n_classes)

mask = np.zeros(test_gt.shape, dtype='bool')
for l in ignored_labels:
    mask[test_gt == l] = True
prediction += 1
prediction[mask] = 0

color_prediction = convert_to_color(prediction)
utils.display_predictions(color_prediction, vis, gt=convert_to_color(test_gt), caption="Prediction vs. test ground truth")

utils.show_results(run_results, vis, label_values=label_values)

Confusion matrix :
[[1939    0    0    0    0    0    7    0    0    0    0    0    0    0
     0    0]
 [1438  906    0    0    0    0 1330    0    0    0    0    0    0    0
     0   15]
 [   0    0  823    0  427    0    0    0  250   36    0    0    0    0
     0    0]
 [   0    0    0 1242   66    0    0    0    0    0    0    0    0    0
     0    0]
 [   0    0    1   13 2318    0    0    0   24    2    2    5    1    0
     0    0]
 [   0    0    0    0    0 3388    0    0    0    3    1    1    2    0
     0    0]
 [   0    0    0    0    0    0 3334   14    0    0    0    4    0    0
     0    6]
 [   0    0    0    0    0    0   18 9800    0  262   39   52    0    0
     4   57]
 [   0    0    0    0    0    0    0    0 5481   48  240    1    0    0
     0    0]
 [   0    0   16    0   11    1    2  327   64 2380   88   72    1    0
     2    0]
 [   0    0    0    0    0    0    0    0  120    9  594   39    0    0
     0    0]
 [   0    0    0    0    0    0    0    0    0

Connection is already closed.
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 11

[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 1

[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 1

[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 1

...

[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 1

[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 1

[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 1

[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 1

[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 111] Connection refused
[Errno 1

In [115]:
results.append(run_results['Accuracy'])

In [116]:
results

[57.472525240213706,
 76.46892140789767,
 81.46813507166571,
 68.33593417506029,
 83.2423547555754]

In [117]:
np.sum(results)/5

73.39757413008256